In [140]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


dataframe_train = pd.read_csv ("train.csv")
dataframe_train.describe()
#dataframe_train.isnull().values.any()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,target
count,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,...,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000
mean,-0.008188,0.046069,0.084133,-0.034631,-0.014021,1.537025,0.115259,-0.086073,0.029376,0.241310,...,-0.097963,0.025496,0.029191,-0.009160,0.003571,0.158798,0.064066,-0.205739,-0.122300,0.287233
std,3.057552,6.858400,2.695550,2.765791,7.261945,0.441206,2.762552,2.742284,2.748676,5.454805,...,2.911122,6.131952,7.884183,2.725234,2.752172,5.803769,2.779181,6.522376,6.802771,0.452503
min,-11.852801,-27.841715,-10.613443,-10.931017,-28.752811,1.000020,-10.311434,-10.075589,-10.141371,-19.277277,...,-10.638970,-28.953492,-33.201209,-11.850828,-9.478779,-20.325866,-9.926252,-28.539398,-28.483072,0.000000
25%,-2.062561,-4.317894,-1.740497,-1.896934,-4.743647,1.189318,-1.703177,-1.950722,-1.846900,-3.445588,...,-2.078737,-3.974946,-5.151041,-1.844369,-1.902487,-3.784460,-1.790703,-4.666848,-4.573261,0.000000
50%,-0.017501,-0.053759,0.084710,-0.075840,-0.091840,1.414150,0.143693,-0.063503,0.063927,0.261190,...,-0.088714,-0.008010,-0.050073,-0.007633,-0.007796,0.166533,0.027659,-0.234126,-0.093245,0.000000
75%,1.991994,4.245097,1.921483,1.777565,4.788682,1.759918,1.964735,1.786275,1.892030,3.906689,...,1.921656,4.007312,5.322672,1.840979,1.833849,4.026855,1.931042,4.296710,4.339118,1.000000
max,12.902967,34.619762,9.492417,11.881847,26.337084,2.717640,10.887146,10.071861,10.213687,21.919386,...,11.498217,26.914131,36.333892,9.047867,11.195738,22.448674,9.925644,27.139111,26.071364,1.000000


In [2]:
#normalized_df=(dataframe_train-dataframe_train.min())/(dataframe_train.max()-dataframe_train.min())
#normalized_dataframe_train['target'] = dataframe_train['target']
#normalized_dataframe_train.describe()
x_trainfull = dataframe_train.drop(['target'], axis=1).values.tolist()
y_trainfull = dataframe_train['target'].values.tolist()

normalized_dataframe_train = dataframe_train
#np.shape(x_train), np.shape(x_train)

In [94]:
#from sklearn.model_selection import train_test_split

#x_train, x_test, y_train, y_test = train_test_split(normalized_dataframe_train.iloc[:,:31], normalized_dataframe_train['target'], shuffle = True , test_size=0,random_state = 1)
#print (x_train.shape, y_train.shape)
#print (x_test.shape, y_test.shape)
#print (x_train.std())
#print (x_test.std())


In [1856]:
#Топология сети

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras import regularizers

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


model=models.Sequential()
model.add(layers.Dropout(0.088, input_shape=(31,)))
model.add(layers.Dense(16,kernel_regularizer=regularizers.l2(0.0017),activation='relu') )
#model.add(layers.Dense(19,kernel_regularizer=regularizers.l2(0.005),activation='relu') )
model.add(layers.Dropout(0.1))
model.add(layers.Dense(16,kernel_regularizer=regularizers.l2(0.0017),activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))


model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_82"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_134 (Dropout)       (None, 31)                0         
                                                                 
 dense_239 (Dense)           (None, 16)                512       
                                                                 
 dropout_135 (Dropout)       (None, 16)                0         
                                                                 
 dense_240 (Dense)           (None, 16)                272       
                                                                 
 dense_241 (Dense)           (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [1816]:
#Кросс-валидация

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=model, epochs=40, batch_size=64, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(pipeline, x_trainfull, y_trainfull, cv=kfold)

print("Model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

INFO:tensorflow:Assets written to: ram:///tmp/tmpfifcbz92/assets
INFO:tensorflow:Assets written to: ram:///tmp/tmpxohmxy7j/assets


KeyboardInterrupt: 

In [1728]:
from imblearn.over_sampling import SMOTE
#smote = SMOTE()
#X_smote, Y_smote = smote.fit_sample(normalized_dataframe_train.iloc[:,:31], normalized_dataframe_train['target'])

print(normalized_dataframe_train.iloc[:,:31].shape)
from imblearn.under_sampling import TomekLinks
tl = TomekLinks(sampling_strategy='majority')
X_tl, Y_tl= tl.fit_sample(normalized_dataframe_train.iloc[:,:31], normalized_dataframe_train['target'])
print(np.shape(X_tl))


smote = SMOTE()
X_smote_tl, Y_smote_tl = smote.fit_sample(X_tl, Y_tl)
print(np.shape(X_smote_tl))


tl = TomekLinks(sampling_strategy='all')
X_smote_tl, Y_smote_tl = tl.fit_sample(X_smote_tl, Y_smote_tl)

print(np.shape(X_smote_tl))


(6963, 31)
(6818, 31)
(9636, 31)
(9630, 31)


In [1893]:
#Обучение модели
from sklearn.model_selection import train_test_split

#x_train, x_test, y_train, y_test = train_test_split(normalized_dataframe_train.iloc[:,:31], normalized_dataframe_train['target'], shuffle = True , test_size=0.2,random_state = 0)
x_train, x_test, y_train, y_test = train_test_split(X_smote_tl, Y_smote_tl, shuffle = True , test_size=0.3,random_state = 3)


from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)
  
#history = model.fit(x_train, y_train, batch_size = 32, 
#                    epochs = 5, verbose=1, validation_data = (x_test, y_test), 
#                    callbacks =[earlystopping])

estimators = []
#estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', model.fit(x_train, y_train, batch_size = 32, 
                    epochs = 1, verbose=1, validation_data = (x_test, y_test), 
                    callbacks =[earlystopping])))
pipeline = Pipeline(estimators)

loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Loss: ", loss)
print("accuracy: ", accuracy)

211/211 [==============================] - 1s 3ms/step - loss: 0.1688 - accuracy: 0.9456 - val_loss: 0.1109 - val_accuracy: 0.9803
Loss:  0.11085505783557892
accuracy:  0.9802699685096741


In [1894]:
#F1score на train.csv
from sklearn import metrics
y_pred = model.predict(x_test, batch_size = 16)
y_pred_bool = (y_pred > .5).astype(int)

metrics.f1_score(y_test, y_pred_bool)

181/181 [==============================] - 0s 2ms/step


0.9799789251844047

In [1891]:
#Запись предсказания модели в файл
dataframe_test = pd.read_csv ("test.csv")
#normalized_df=(dataframe_test-dataframe_test.min())/(dataframe_test.max()-dataframe_test.min())
#normalized_dataframe_test = dataframe_test

x_pred = dataframe_test.values.tolist()
y_pred = model.predict(x_pred)
Y = (y_pred > .5).astype(int)
np.savetxt('pred.csv',Y, fmt='%d',header='target')

123/123 [==============================] - 0s 2ms/step


In [1892]:
#F1score относительно лучшего решения
y_pred = np.loadtxt('pred.csv', skiprows = 1)
y_best = np.loadtxt('predbest.csv', skiprows = 1)
sum1 = 0
for i in range(len(y_pred)):
    sum1 += abs(y_best[i] - y_pred[i])
print(np.shape(y_pred), np.shape(y_best))
print(sum1)
metrics.f1_score(y_best, y_pred)

(3920,) (3920,)
551.0


0.8580994076744785

In [1367]:
####

In [316]:
from RadViz.main import RadViz2D
from RadViz.main import RadViz3D
import time
import plotly.io as pio
pio.renderers.default='iframe'

BPs=10000
time_start = time.time()
y=dataframe_train['target']
X=dataframe_train.drop(['target'], axis=1)
RadViz3D(y,X,BPs)
print( 'RadViz3D done! Time elapsed: {} seconds'.format(time.time()-time_start))

RadViz3D done! Time elapsed: 0.22623157501220703 seconds


In [62]:

BPs=10000
time_start = time.time()
X = X_smote_tl
y = Y_smote_tl 
RadViz3D(y,X,BPs)
print( 'RadViz3D done! Time elapsed: {} seconds'.format(time.time()-time_start))

RadViz3D done! Time elapsed: 0.19585466384887695 seconds


In [330]:
BPs=10000
time_start = time.time()
dataframe_test['target'] = Y
y=dataframe_test['target']
X=dataframe_test.drop(['target'], axis=1)
RadViz3D(y,X,BPs)
print( 'RadViz3D done! Time elapsed: {} seconds'.format(time.time()-time_start))

RadViz3D done! Time elapsed: 0.17314839363098145 seconds
